In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import glob
import os
import re
import sys
import xarray as xr
import rioxarray as rio
sys.path.append(os.path.abspath("c:\\Users\\m1865\\Desktop\\GIS Thesis Python"))
import mySTD
# Data directory of ARPA data
dd_arpa = "c:\\Users\\m1865\\Desktop\\GIS Thesis Python\\ARPA"
dd_cams = "c:\\Users\\m1865\\Desktop\\GIS Thesis Python\\CAMS"
dd_main = "c:\\Users\\m1865\\Desktop\\GIS Thesis Python"
# Set current working directory
cwd = os.getcwd()
cwd

In [ ]:
myFile_NO2 = []
myFile_O3 = []
myFile_PM25 = []
myFile_PM10 = []
# Filter all the file names of the data from the data folder
for filename in os.listdir(dd_cams):
    match = re.match(r'(.*)2021(.*).nc', filename)
    # print('Check Match')
    if match:
        # print('Matched!')
        if match.group(1) == "NO2":
            myFile_NO2.append(filename)
        elif match.group(1) == "ozone":
            myFile_O3.append(filename)
        elif match.group(1) == "PM25": 
            myFile_PM25.append(filename)
        elif match.group(1) == "PM10":
            myFile_PM10.append(filename)

In [ ]:
myList_NO2 = []
myList_O3 = []
myList_PM10 = []
myList_PM25 = []
for filename in myFile_NO2:
    ds = xr.open_dataset(dd_cams + "\\" + filename)
    newlong = mySTD.Lon360to180(ds)
    newtime = mySTD.TimetoDate64(ds)
    ds = ds.assign_coords(time=newtime,longitude=newlong).sortby("longitude")
    # print(ds.time.dt.month)
    myList_NO2.append(ds)
for filename in myFile_O3:
    ds = xr.open_dataset(dd_cams + "\\" + filename)
    newlong = mySTD.Lon360to180(ds)
    newtime = mySTD.TimetoDate64(ds)
    ds = ds.assign_coords(time=newtime,longitude=newlong).sortby("longitude")
    myList_O3.append(ds)
for filename in myFile_PM10:
    ds = xr.open_dataset(dd_cams + "\\" + filename)
    newlong = mySTD.Lon360to180(ds)
    newtime = mySTD.TimetoDate64(ds)
    ds = ds.assign_coords(time=newtime,longitude=newlong).sortby("longitude")
    myList_PM10.append(ds)
for filename in myFile_PM25:
    ds = xr.open_dataset(dd_cams + "\\" + filename)
    newlong = mySTD.Lon360to180(ds)
    newtime = mySTD.TimetoDate64(ds)
    ds = ds.assign_coords(time=newtime,longitude=newlong).sortby("longitude")
    myList_PM25.append(ds)

In [ ]:
# Check
myList_NO2[1]

In [ ]:
myList_NO2_Mean = []
myList_O3_Mean = []
myList_PM10_Mean = []
myList_PM25_Mean = []
for i in range(len(myList_NO2)):
    ds = myList_NO2[i]
    ds = ds.mean(dim='time')
    ds = ds.mean(dim='level')
    ds = ds.assign_attrs({"month": i+1})
    myList_NO2_Mean.append(ds)
for i in range(len(myList_O3)):
    ds = myList_O3[i]
    ds = ds.mean(dim='time')
    ds = ds.mean(dim='level')
    ds = ds.assign_attrs({"month": i+1})
    myList_O3_Mean.append(ds)
for i in range(len(myList_PM10)):
    ds = myList_PM10[i]
    ds = ds.mean(dim='time')
    ds = ds.mean(dim='level')
    ds = ds.assign_attrs({"month": i+1})
    myList_PM10_Mean.append(ds)
for i in range(len(myList_PM25)):
    ds = myList_PM25[i]
    ds = ds.mean(dim='time')
    ds = ds.mean(dim='level')
    ds = ds.assign_attrs({"month": i+1})
    myList_PM25_Mean.append(ds)

In [ ]:
# Check
myList_PM25_Mean[4]

In [ ]:
lom = gpd.read_file('..\lom.shp')

In [ ]:
myList_NO2_Clipped = []
myList_O3_Clipped = []
myList_PM10_Clipped = []
myList_PM25_Clipped = []
for i in range(len(myList_NO2_Mean)):
    ds = myList_NO2_Mean[i]
    ds.rio.write_crs(4326, inplace=True)
    ds = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_NO2_Clipped.append(ds)
for i in range(len(myList_O3_Mean)):
    ds = myList_O3_Mean[i]
    ds.rio.write_crs(4326, inplace=True)
    ds = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_O3_Clipped.append(ds)
for i in range(len(myList_PM10_Mean)):
    ds = myList_PM10_Mean[i]
    ds.rio.write_crs(4326, inplace=True)
    ds = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_PM10_Clipped.append(ds)
for i in range(len(myList_PM25_Mean)):
    ds = myList_PM25_Mean[i]
    ds.rio.write_crs(4326, inplace=True)
    ds = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_PM25_Clipped.append(ds)


In [ ]:
print(myList_NO2_Clipped[0].mean())
myList_NO2_Clipped[0]

In [ ]:
# Upscale CAMS
myList_NO2_Clipped_Up = []
myList_O3_Clipped_Up = []
myList_PM10_Clipped_Up = []
myList_PM25_Clipped_Up = []
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
for i in range(len(myList_NO2_Clipped)):
    ds = myList_NO2_Clipped[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.no2_conc,geometry=gpd.points_from_xy(ds.longitude,ds.latitude))
    ds = ds.dropna()
    ds = make_geocube(
        vector_data=ds,
        measurements=['no2_conc'],
        resolution=(-0.01, 0.01),
        rasterize_function=rasterize_points_griddata,
    )
    ds.rio.write_crs(4326, inplace=True)
    upscale_clip = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_NO2_Clipped_Up.append(upscale_clip)
for i in range(len(myList_O3_Clipped)):
    ds = myList_O3_Clipped[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.o3_conc,geometry=gpd.points_from_xy(ds.longitude,ds.latitude))
    ds = ds.dropna()
    ds = make_geocube(
        vector_data=ds,
        measurements=['o3_conc'],
        resolution=(-0.01, 0.01),
        rasterize_function=rasterize_points_griddata,
    )
    ds.rio.write_crs(4326, inplace=True)
    upscale_clip = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_O3_Clipped_Up.append(upscale_clip)
for i in range(len(myList_PM10_Clipped)):
    ds = myList_PM10_Clipped[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.pm10_conc,geometry=gpd.points_from_xy(ds.longitude,ds.latitude))
    ds = ds.dropna()
    ds = make_geocube(
        vector_data=ds,
        measurements=['pm10_conc'],
        resolution=(-0.01, 0.01),
        rasterize_function=rasterize_points_griddata,
    )
    ds.rio.write_crs(4326, inplace=True)
    upscale_clip = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_PM10_Clipped_Up.append(upscale_clip)
for i in range(len(myList_PM25_Clipped)):
    ds = myList_PM25_Clipped[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.pm2p5_conc,geometry=gpd.points_from_xy(ds.longitude,ds.latitude))
    ds = ds.dropna()
    ds = make_geocube(
        vector_data=ds,
        measurements=['pm2p5_conc'],
        resolution=(-0.01, 0.01),
        rasterize_function=rasterize_points_griddata,
    )
    ds.rio.write_crs(4326, inplace=True)
    upscale_clip = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_PM25_Clipped_Up.append(upscale_clip)

In [ ]:
myList_PM25_Clipped_Up[0]

In [ ]:
cams_gdf_NO2 = []
for i in range(len(myList_NO2_Clipped_Up)):
    ds = myList_NO2_Clipped_Up[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.no2_conc,geometry=gpd.points_from_xy(ds.x,ds.y))
    ds = ds.dropna()
    ds = ds.set_crs('epsg:4326')
    ds = ds.to_crs(32632)
    ds['EST_UTM'] = ds.geometry.x
    ds['NORD_UTM'] = ds.geometry.y
    ds = ds.round({'EST_UTM': -3, 'NORD_UTM': -3})
    ds = ds.drop(columns={'geometry'})
    ds['Month'] = i+1
    ds = pd.DataFrame(ds)
    cams_gdf_NO2.append(ds)


In [ ]:
cams_gdf_NO2[0]

In [ ]:
cams_gdf_O3 = []
for i in range(len(myList_O3_Clipped_Up)):
    ds = myList_O3_Clipped_Up[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.o3_conc,geometry=gpd.points_from_xy(ds.x,ds.y))
    ds = ds.dropna()
    ds = ds.set_crs('epsg:4326')
    ds = ds.to_crs(32632)
    ds['EST_UTM'] = ds.geometry.x
    ds['NORD_UTM'] = ds.geometry.y
    ds = ds.round({'EST_UTM': -3, 'NORD_UTM': -3})
    ds = ds.drop(columns={'geometry'})
    ds['Month'] = i+1
    ds = pd.DataFrame(ds)
    cams_gdf_O3.append(ds)

In [ ]:
cams_gdf_PM10 = []
for i in range(len(myList_PM10_Clipped_Up)):
    ds = myList_PM10_Clipped_Up[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.pm10_conc,geometry=gpd.points_from_xy(ds.x,ds.y))
    ds = ds.dropna()
    ds = ds.set_crs('epsg:4326')
    ds = ds.to_crs(32632)
    ds['EST_UTM'] = ds.geometry.x
    ds['NORD_UTM'] = ds.geometry.y
    ds = ds.round({'EST_UTM': -3, 'NORD_UTM': -3})
    ds = ds.drop(columns={'geometry'})
    ds['Month'] = i+1
    ds = pd.DataFrame(ds)
    cams_gdf_PM10.append(ds)

In [ ]:
cams_gdf_PM25 = []
for i in range(len(myList_PM25_Clipped_Up)):
    ds = myList_PM25_Clipped_Up[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.pm2p5_conc,geometry=gpd.points_from_xy(ds.x,ds.y))
    ds = ds.dropna()
    ds = ds.set_crs('epsg:4326')
    ds = ds.to_crs(32632)
    ds['EST_UTM'] = ds.geometry.x
    ds['NORD_UTM'] = ds.geometry.y
    ds = ds.round({'EST_UTM': -3, 'NORD_UTM': -3})
    ds = ds.drop(columns={'geometry'})
    ds['Month'] = i+1
    ds = pd.DataFrame(ds)
    cams_gdf_PM25.append(ds)

In [ ]:
cams_NO2 = pd.concat(cams_gdf_NO2)
cams_O3 = pd.concat(cams_gdf_O3)
cams_PM10 = pd.concat(cams_gdf_PM10)
cams_PM25 = pd.concat(cams_gdf_PM25)
cams_NO2

In [ ]:
cams_NO2.to_csv('CAMS NO2 Upscale.csv', index=False)
cams_O3.to_csv('CAMS O3 Upscale.csv', index=False)
cams_PM10.to_csv('CAMS PM10 Upscale.csv', index=False)
cams_PM25.to_csv('CAMS PM25 Upscale.csv', index=False)

In [ ]:
# cams_ALL_List = [cams_NO2, cams_O3, cams_PM10, cams_PM25]
# cams_ALL = pd.concat(cams_ALL_List)
# cams_ALL